In [2]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [101]:
# create mathy column headers 
hsa_dataset = pd.read_excel("https://ndownloader.figshare.com/files/5324491", None)
data = hsa_dataset["All"]
data.rename(columns = {'x':'$x$'}, inplace = True)

In [102]:
labels = data["lig_contact"] != "not_displaced"
data.insert(1, "category", labels.values)
data["category"] = data.category.astype(int)

In [103]:
data.head()

,Protein,category,index,type,$x$,y,z,occ,E_sw,E_ww,...,E^nbr_ww,N^nbrs,f_enc,N^HB_sw,N^HB_ww,f^HB_ww,"N^HB_ww,lost",lig_contact,t_statistic,p_value
0,abl,0,0,C2F,-1.90,-12.30,-0.79,0.9234,-7.886070,-2.330430,...,-1.237988,2.363223,0.549862,1.595408,1.367122,57.849876,1.831040,not_displaced,39.731908,0.000000e+00
1,abl,0,1,C2F,-5.63,-4.61,-4.35,0.9246,-9.815779,-0.801470,...,-1.119749,2.431322,0.537030,1.587822,1.556781,64.030249,1.788310,not_displaced,57.148045,0.000000e+00
2,abl,1,2,C1F,-3.59,-9.44,-1.87,0.9148,-7.585982,-4.115125,...,-1.462429,2.996174,0.429581,1.879974,1.990162,66.423438,1.430505,hydrophilic,117.529066,0.000000e+00
3,abl,0,3,C2F,-9.49,-4.17,-5.07,0.9068,-10.545356,0.613647,...,-0.620995,3.208866,0.391129,1.483569,1.650088,51.422778,1.302460,not_displaced,21.260681,5.495392e-99
4,abl,0,4,P1U,1.31,-8.94,-6.30,0.8894,-4.102407,-5.039672,...,-2.175132,1.981785,0.622517,0.945919,1.753429,88.477250,2.072982,not_displaced,-22.268783,1.752911e-108


In [1]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler

In [125]:
data = data.dropna(axis=0)
X, y = data.iloc[:, 7:-3], data.iloc[:, 1]

In [131]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [132]:
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.fit_transform(X_test)

In [134]:
X_train_std.shape, X_test.shape

((173, 11), (44, 11))

In [146]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

In [147]:
model = LogisticRegression(C=100.0, random_state=1)
model.fit(X_train_std, y_train)
model.predict(X_test_std)
y_scores = model.predict_proba(X_test_std)
print(y_scores)

[[ 0.63631811  0.36368189]
 [ 0.81934902  0.18065098]
 [ 0.47958109  0.52041891]
 [ 0.48124079  0.51875921]
 [ 0.51397856  0.48602144]
 [ 0.42167251  0.57832749]
 [ 0.60581214  0.39418786]
 [ 0.52062976  0.47937024]
 [ 0.77652965  0.22347035]
 [ 0.62836067  0.37163933]
 [ 0.77104924  0.22895076]
 [ 0.4395446   0.5604554 ]
 [ 0.55620158  0.44379842]
 [ 0.60356076  0.39643924]
 [ 0.6465952   0.3534048 ]
 [ 0.55261693  0.44738307]
 [ 0.67350815  0.32649185]
 [ 0.25961181  0.74038819]
 [ 0.4064833   0.5935167 ]
 [ 0.58798678  0.41201322]
 [ 0.52701002  0.47298998]
 [ 0.3334739   0.6665261 ]
 [ 0.39002504  0.60997496]
 [ 0.30341132  0.69658868]
 [ 0.85488878  0.14511122]
 [ 0.57769633  0.42230367]
 [ 0.29780796  0.70219204]
 [ 0.38458232  0.61541768]
 [ 0.29843152  0.70156848]
 [ 0.72359857  0.27640143]
 [ 0.38546085  0.61453915]
 [ 0.77676168  0.22323832]
 [ 0.45001278  0.54998722]
 [ 0.53133115  0.46866885]
 [ 0.59425877  0.40574123]
 [ 0.55442115  0.44557885]
 [ 0.24898551  0.75101449]
 

In [151]:
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 0]) 

In [153]:
cross_val_score(model, X_train_std, y_train, cv=3, scoring="accuracy")

array([ 0.68965517,  0.60344828,  0.57894737])

In [ ]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
scores = []
for train, test in kfold.split(X_tr, Y):
    model = Sequential()
    model.add(Dense(64, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dropout(0.5))
    #model.add(Dense(64, activation='relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=1000, batch_size=128)
    score = model.predict_proba(X[test])
    scores.append((roc_auc_score(Y[test], score)))

print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

fpr, tpr, thresholds = roc_curve(Y[test], score)
plot_roc_curve([fpr], [tpr], "ANNDescriptors", ["Descriptors"])
